# Ensemble Time Series

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import timeit

import warnings
warnings.filterwarnings("ignore")

# List of Stocks and ETFs
Provided by Thomas Choi.

In [4]:
stock_list = pd.read_csv("../stocks_and_etfs/stock_list.csv")
etf_list = pd.read_csv("../stocks_and_etfs/etf_list.csv")

In [5]:
import random
# Choose a random stock
#stock_symbol = random.choice(stock_list.iloc[:,0])
stock_symbol = "FCX"
print(stock_symbol)

FCX


## MySQL connection
Choosing one stock from SQL query to reduce query time.

In [6]:
import os
import sys
import mysql.connector

HOST="143.244.188.157"
PORT="3306"
USER="patrick-finProj"
PASSWORD="Pat#21$rick"

try: 
    conn = mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database="GlobalMarketData"
    )
    query = f"SELECT Date, Close, Open, High, Low, Volume from histdailyprice3 WHERE Symbol='{stock_symbol}';"
    histdailyprice3 = pd.read_sql(query, conn)
    conn.close()
except Exception as e:
    conn.close()
    print(str(e))

# Data

In [7]:
df = histdailyprice3.copy()
df.head()

,Date,Close,Open,High,Low,Volume
0,1998-01-01,7.875,7.875,7.875,7.875,0
1,1998-01-02,7.875,8.000,7.875,7.970,279300
2,1998-01-05,7.970,8.000,7.845,7.875,455300
3,1998-01-06,7.845,7.875,7.750,7.750,476200
4,1998-01-07,7.720,7.875,7.685,7.845,464100


# Normalize

In [9]:
from sklearn.preprocessing import MinMaxScaler
# normalize data
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(df["Close"].values.reshape(-1, 1))

# Train-Test Split

In [10]:
prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x - prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [12]:
!pip install tensorflow

  Using cached tensorflow-2.7.0-cp38-cp38-win_amd64.whl (430.8 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Patrick.Guo\\.conda\\envs\\rtx3000s\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



# Build LSTM Model

In [11]:
from keras import Sequential
from keras.layers import Dense, LSTM

def LSTM_model():
    
    model = Sequential()
    
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=1))
    
    return model

ImportError: cannot import name 'StructureCoder' from 'tensorflow.python.saved_model.nested_structure_coder' (C:\Users\Patrick.Guo\.conda\envs\rtx3000s\lib\site-packages\tensorflow\python\saved_model\nested_structure_coder.py)

# Callbacks
**EarlyStoping:** It will stop the traning if score of model didn't increase. This prevent model from overfitting. We are to set max in 10 epoches if it didn't increase then we will stop the training <br>
**ReduceLROnPlateau:** Use for reduce the learning rate. In 3 steps the score didn't increase we will reduce the learning rate to improve the training <br>
**ModelCheckpoint:** Use for save model only when the score increased

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, 
                    save_weights_only=True)
]

# Compile LSTM Model

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics=['accuracy'])

In [ ]:
start = timeit.default_timer()

model.fit(X_train, y_train, 
          epochs=100, batch_size=32, 
          validation_data=(X_valid, y_valid), callbacks=callbacks)

stop = timeit.default_timer()
print('Time: ', stop - start)

# Prediction

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

# Time the grid search
start = timeit.default_timer()

model = Sequential()
# Identifying the number of hidden neurons, the activation function,  and input shape
model.add(LSTM(512, activation='relu', 
               input_shape=(n_steps, n_features), 
               return_sequences=True))
# Adding a dropout layer to prevent overfitting
model.add(Dropout(0.2))
# Adding one more hidden layer with 256 neurons, with a relu activation function
model.add(LSTM(256,activation='relu'))
# Flattening the model to vectorize the three-dimensional matrix
model.add(Flatten())
# Adding an output layer with a linear activation function
model.add(Dense(1, activation='linear'))
# Compiling LSTM with Root Mean Square Propagation and mean squared error (MSE)
model.compile(optimizer='rmsprop', loss='mean_squared_error',
                       metrics=['mse'])

stop = timeit.default_timer()
print('Time: ', stop - start)

# Visualize prediction

In [ ]:
predicted_value = model.predict(X_test)

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(predicted_value, color= 'red')
plt.plot(Y_test, color='green')
plt.title("Close price of stocks sold")
plt.xlabel("Time")
plt.ylabel("Stock Opening Price")
plt.show()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=400, batch_size=150, verbose=0,
                    validation_split = 0.10)

In [ ]:
start = X_train[X_train.shape[0] - 13]
x_input = start
x_input = x_input.reshape((1, n_steps, n_features))

In [ ]:
tempList = []
for i in range(len(diff_test)):
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    x_input = np.append(x_input, yhat)
    x_input = x_input[1:]
    tempList.append(yhat)

In [ ]:
fig, ax = plt.subplots()
#ax.plot(diff_test, label='Actual Stock Price', linestyle='--')
ax.plot(np.array(tempList).flatten(),
        linestyle='solid', label="Prediction")
ax.set_title('Predicted Stock Price')
ax.legend(loc='best')
plt.show()